# Advent of Code 2023

https://adventofcode.com/2023

In [1]:
# A few standard library imports
import itertools, math, collections, os, functools, json, time, re, bisect
# nice printing
from IPython.display import display, clear_output, HTML

In [2]:
def getData(day, year=2023):
    if not os.path.exists("data-%d" %year):
        os.mkdir("data-%d" %year)
    if os.path.exists("data-%d/day%dinput.txt" %(year, day)):
        with open("data-%d/day%dinput.txt" %(year, day), 'r') as f:
            return f.read()
    import browsercookie, urllib.request
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(browsercookie.firefox()))
    assert b'watercrossing' in opener.open("https://adventofcode.com/%d/" %year).read()
    data = opener.open("https://adventofcode.com/%d/day/%d/input" %(year, day)).read().decode('utf-8')
    with open("data-%d/day%dinput.txt" %(year, day), 'w') as f:
        f.write(data)
    return data

## Day 1

In [3]:
day1data = getData(1).split()

In [4]:
sum(int(next(c for c in x if 47 < ord(c) < 58) + next(c for c in x[::-1] if 47 < ord(c) < 58)) for x in day1data)

54927

In [5]:
z = list(zip(["one", "two", "three", "four", "five", "six", "seven", "eight", "nine"] + [str(x) for x in range(0, 10)], 
              [str(x) for x in range(1, 10)] + [str(x) for x in range(0, 10)]))

In [6]:
sum(int(min([(x.find(c), d) for c, d in z if c in x], key=lambda a: a[0])[1] + 
        min([(x[::-1].find(c[::-1]), d) for c, d in z if c in x], key=lambda a: a[0])[1]) for x in day1data)

54581